In [1]:
import os

In [2]:
%pwd

'd:\\VS Code\\mydlproject\\Kidney_DL_project\\research'

In [3]:
os.getcwd()

'd:\\VS Code\\mydlproject\\Kidney_DL_project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\VS Code\\mydlproject\\Kidney_DL_project'

1. Update config.yaml
2. Update secrets.yaml [Optional]
3. Update params.yaml
4. Update the entity
5. Update the configuration manager in src config
6. Update the components
7. Update the pipeline 
8. Update the main.py

In [6]:
# update config.yaml

In [7]:
# update params.yaml

In [8]:
# Update the entity

In [9]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [10]:
# Update the configuration manager in src config

In [11]:
from src.cnnClassifier.utils.comman import read_yaml, create_directories
from src.cnnClassifier.constants import *

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [13]:
# Update the Component

In [14]:
import os
import zipfile
import gdown
from src.cnnClassifier import logger
from src.cnnClassifier.utils.comman import get_size

In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
# Update the pipeline

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-05 17:22:29,453: INFO: comman: yaml file: config\config.yaml loaded successfully]
[2025-05-05 17:22:29,454: INFO: comman: yaml file: params.yaml loaded successfully]
[2025-05-05 17:22:29,456: INFO: comman: created directory at: artifacts]
[2025-05-05 17:22:29,457: INFO: comman: created directory at: artifacts/data_ingestion]
[2025-05-05 17:22:29,457: INFO: 986331348: Downloading data from https://drive.google.com/file/d/1LIc7V18n5ZWXFIfnT8uE6AQjmzyDvU7l/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1LIc7V18n5ZWXFIfnT8uE6AQjmzyDvU7l
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1LIc7V18n5ZWXFIfnT8uE6AQjmzyDvU7l&confirm=t&uuid=86d0aa0c-566a-4545-b014-5e0d6bb63712
To: d:\VS Code\mydlproject\Kidney_DL_project\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:09<00:00, 5.95MB/s]

[2025-05-05 17:22:44,653: INFO: 986331348: Downloaded data from https://drive.google.com/file/d/1LIc7V18n5ZWXFIfnT8uE6AQjmzyDvU7l/view?usp=sharing into file artifacts/data_ingestion/data.zip]
